In [1]:
import json
import itertools
from fractions import Fraction
import numpy as np
import pandas

#GUI elements
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

In [2]:
%run logic.ipynb
%run participant.ipynb


----------------------------------------------------------------------
Ran 0 tests in 0.000s

OK


In [3]:
answer_dist = {}

In [4]:
def update_dist(change):
    
    current_accordion = dist_accordion.children[dist_accordion.selected_index]
    dropdown = current_accordion.children[1]
    active_sliders = current_accordion.children[2].children
    
    if dropdown.value != 'Custom':
        shape_slider = current_accordion.children[3]

        # Draw values from dist
        values = distMap[dropdown.value](len(active_sliders),shape_slider.value)

        for slider in active_sliders:

            index = active_sliders.index(slider)
            slider.value = np.round(values[index], decimals=sliders_precision)
            
     
    #Save values
    for slider in active_sliders:
        index = active_sliders.index(slider)
        answer_dist[dist_accordion.selected_index][index][dropdown.value] = slider.value

        

In [5]:
def dropdown_action(change):
    current_accordion = dist_accordion.children[dist_accordion.selected_index]
    dropdown = current_accordion.children[1]
    shape_slider = current_accordion.children[3]
    shape_slider.hold_trait_notifications()

    # Update the shape slider according to the chosen distribution
    if(dropdown.value is 'Uniform' or dropdown.value is 'Custom'):
        shape_slider.disabled=True
        shape_slider.layout.visibility='hidden'
    else:

        if(dropdown.value is 'Geom'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1.
            shape_slider.value = 0.5
        elif(dropdown.value is 'Logser'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1. - shape_slider.step
            shape_slider.value = 0.6
        elif(dropdown.value is 'Planck'):
            shape_slider.step = 0.01
            shape_slider.min = 0. + shape_slider.step
            shape_slider.max = 1.
            shape_slider.value = 0.51
        elif(dropdown.value is 'Zipf'):
            shape_slider.max = 10.
            shape_slider.step = 0.01
            shape_slider.min = 1. + shape_slider.step
            shape_slider.value = 1.5

        shape_slider.disabled=False
        shape_slider.layout.visibility='visible'
        
    #Reset values
    active_sliders = current_accordion.children[2].children   
     
    for slider in active_sliders:
        index = active_sliders.index(slider)
        
        val = dropdown.value
        if val not in answer_dist[dist_accordion.selected_index][index].keys():
            update_dist(None)
        slider.value = answer_dist[dist_accordion.selected_index][index][val]

In [6]:
answer_dist

{}

In [7]:
#Example JSON
root_question = {'qid':0,
'question': 'Blood type?',
'answers': ['O+','A+','Other'],
'probability': ['1/3','1/3','1/3']}

follow_up_1 = {'qid':1,
'question': 'Specify...',
'answers': ['B+','AB+','O-','A-','Other'],
'probability': ['1/5','1/5','1/5','1/5','1/5']}

follow_up_2 = {'qid':2,
'question': 'Specify further...',
'answers': ['B-','AB-'],
'probability': ['1/2','1/2']}

paths = [[0,'Other',1], [1,'Other',2]]

json_poll = {
    'roots': [root_question],
    'children': [follow_up_1, follow_up_2],
    'paths':paths,
    'truth':['1/4']
}
json_poll

{'children': [{'answers': ['B+', 'AB+', 'O-', 'A-', 'Other'],
   'probability': ['1/5', '1/5', '1/5', '1/5', '1/5'],
   'qid': 1,
   'question': 'Specify...'},
  {'answers': ['B-', 'AB-'],
   'probability': ['1/2', '1/2'],
   'qid': 2,
   'question': 'Specify further...'}],
 'paths': [[0, 'Other', 1], [1, 'Other', 2]],
 'roots': [{'answers': ['O+', 'A+', 'Other'],
   'probability': ['1/3', '1/3', '1/3'],
   'qid': 0,
   'question': 'Blood type?'}],
 'truth': ['1/4']}

In [8]:
%run gui_components.ipynb

<IPython.core.display.Javascript object>

In [9]:
# Distribution sliders per question (non-flattened)
children = []

subtrees = pollToSubtrees(json_poll)
for question in subtrees:
    for subquestion in subtrees[question]:
        dist_accordion.set_title(subquestion['qid'], 'Question: '+subquestion['question'])
        html = widgets.HTML('<h3>Choose weight distribution...</h3>')
        to_add = []
        answer_dist[subquestion['qid']] = {}
        
        count = 0
        for answer in subquestion['answers']:
            probability = subquestion['probability'][count]
            to_add.append(widgets.FloatSlider(
                value=probability,
                min=0,
                max=1.0,
                step=0.01,
                description=answer,
                disabled=False,
                continuous_update=False,
                orientation='vertical',
                readout=True,
                readout_format='.1f',
            ))
            answer_dist[subquestion['qid']][count] = {'Custom': probability}
            count +=1
        
        #Components
        _shape_slider = widgets.FloatSlider(min=0., max=5., step=0.1, value=0., continuous_update=False, description='Shape', disabled=True) 
        _dropdown = widgets.Dropdown(options=dists, value='Custom', description='Distribution:', disabled=False)
        _button = widgets.Button(description='Update weights', icon='fa-bar-chart', button_style='success')
        
        #Actions
        _button.on_click(update_dist)
        _dropdown.observe(dropdown_action)
        
        contents =[html, _dropdown, widgets.HBox(to_add),_shape_slider, _button]
        children.append( widgets.VBox(contents))
        
dist_accordion.children = children
dist_box = widgets.VBox([dist_accordion])

In [10]:
display(widgets.HTML('<center><h1> Choose Parameters'))
display(start_tab)

HTML(value=u'<center><h1> Choose Parameters')

VGFiKGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J0lucHV0IEpTT046JyksIFNlbGVjdChvcHRpb25zPSgnYmxvb2QuanNvbicsKSwgdmFsdWXigKY=


In [11]:
display(widgets.HTML('<center><h1> Set <i>True</i> Answer Distribution</h1></center>'))
display(dist_accordion)
dropdown_action(None)

HTML(value=u'<center><h1> Set <i>True</i> Answer Distribution</h1></center>')

QWNjb3JkaW9uKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIVE1MKHZhbHVlPXUnPGgzPkNob29zZSB3ZWlnaHQgZGlzdHJpYnV0aW9uLi4uPC9oMz4nKSwgRHJvcGRvd24oZGVzY3JpcHRpb27igKY=


In [12]:
matrices = pollToMatrix(json_poll)
epsilons = pollToEpsilon(json_poll)
ps = pollToPaths(json_poll)

In [13]:
selected = {}
for question in subtrees:
    selected[question] = {}
    for subquestion in subtrees[question]:
        selected[question][subquestion['qid']] = []
        current_box = dist_accordion.children[subquestion['qid']].children
        current_dropdown = current_box[1]
        current_sliders = current_box[2].children
  
        for slider in current_sliders:
            index = current_sliders.index(slider)
            selected[question][subquestion['qid']].append(answer_dist[subquestion['qid']][index][current_dropdown.value])
            
#NORMALIZE SLIDER VALUES BEFORE MOVING ON

# Find matching path
# {qid: (Path, Fraction)}


In [14]:
# Turn slides to JSON, then parse to find path-weight pairs

#FIXME: Don't overwrite actual json_poll, do deep copy!
 
dist_poll = json_poll

for question in subtrees:
    root = filter(lambda x: x['qid']==question, dist_poll['roots'])[0]
    root['probability'] = selected[question][question]
    
    for subquestion in subtrees[question]:
        # Roots have already been added, will also return empty list so ignore
        question_struct = filter(lambda x: x['qid']==subquestion['qid'], dist_poll['children'])
        if len(question_struct) > 0:
            question_struct[0]['probability'] = selected[question][subquestion['qid']]
            
    # FIXME: NEEDS TO SUM TO 1.
    # NORMALIZE VALUES!

            
weights = pollToPathWeights(dist_poll)

In [15]:
ans_dist_zip = {}
for question in ps.keys():
    paths = ps[question]
    weight = weights[question]
    zipped = []
    for path in paths:
        zipped.append((path, weights[question][pathToKey(path)]))

    ans_dist_zip[question] = zipped
ans_dist_zip

{0: [(['O+'], Fraction(1, 3)),
  (['A+'], Fraction(1, 3)),
  (['Other', 'B+'], Fraction(1, 15)),
  (['Other', 'AB+'], Fraction(1, 15)),
  (['Other', 'O-'], Fraction(1, 15)),
  (['Other', 'A-'], Fraction(1, 15)),
  (['Other', 'Other', 'B-'], Fraction(1, 30)),
  (['Other', 'Other', 'AB-'], Fraction(1, 30))]}

In [16]:

participants = []
n = 100
participant_budget = 40

#TODO: Loop x times for confidence

raw_responses = []
real_answers = []

for i in range(0, n):
    new_participant = Participant(participant_budget, json_poll)
    new_participant.setMatrices(matrices)
    new_participant.setEpsilons(epsilons)
    new_participant.createAnswer(ans_dist_zip)

    real_answers.append(dict(new_participant.getAnswer())) #Beware: deep copy required!
    raw_responses.append(new_participant.randomizedResponse())
    
    participants.append(new_participant)

In [17]:
# Count occurences

In [18]:
lists_responses = {}
lists_answer = {}
for question in subtrees.keys():
    lists_responses[question] = map(lambda x: pathToKey(x[question]), raw_responses)
    lists_answer[question] = map(lambda x: pathToKey(x[question]), real_answers)

In [19]:
ps[question]

[['O+'],
 ['A+'],
 ['Other', 'B+'],
 ['Other', 'AB+'],
 ['Other', 'O-'],
 ['Other', 'A-'],
 ['Other', 'Other', 'B-'],
 ['Other', 'Other', 'AB-']]

In [20]:
response_frame = pd.DataFrame(lists_responses)
response_frequency = {}
answer_frame = pd.DataFrame(lists_answer)
answer_frequency = {}

for question in subtrees.keys():
    response_frequency[question] = {}
    answer_frequency[question] = {}

    to_match = pd.unique(response_frame[question])
    for match in to_match:
        response_frequency[question][match] = len(response_frame[response_frame[question]==match])
        answer_frequency[question][match] = len(answer_frame[answer_frame[question]==match])
        
        
response_frequency, answer_frequency

({0: {'A+': 30,
   'O+': 38,
   'OtherA-': 7,
   'OtherAB+': 2,
   'OtherB+': 11,
   'OtherO-': 7,
   'OtherOtherAB-': 1,
   'OtherOtherB-': 4}},
 {0: {'A+': 32,
   'O+': 40,
   'OtherA-': 5,
   'OtherAB+': 7,
   'OtherB+': 5,
   'OtherO-': 8,
   'OtherOtherAB-': 2,
   'OtherOtherB-': 1}})

In [21]:
# BAYES THEOREM

In [22]:

#TODO: compare to ground truth

#TODO: display graphs